# Data preprocessing for Textmining

____

금일 처리해야하는 사항들 

* feed 내에서 나온 comment들을 모아야 합니다.
> output: [댓글, [대댓글,대댓글,대댓글]] 의 형태로 나타나게끔 묶어줍니다.

In [279]:
# 필요한 package 
import pandas as pd 
import ast #str to list
from tqdm import tqdm 
import tqdm
import numpy as np

In [96]:

def read_data():
    hyatt= pd.read_csv("hyatt_comment",index_col=[0]).reset_index()
    hilton=pd.read_csv("hilton_comment",index_col=[0]).reset_index()
    holiday=pd.read_csv("holiday_comment",index_col=[0]).reset_index()
    wyndham=pd.read_csv("wyndham_comment",index_col=[0]).reset_index()
    
    return (hilton,hyatt,holiday,wyndham)
hilton,hyatt,holiday,wyndham=read_data()
hilton.head()
test=hilton[:30]

In [97]:
# comment
def string_to_list(df,columns):
    for col in columns:
        df[col]=df[col].apply(lambda x: ast.literal_eval(x))
        
for hotel in (hilton,hyatt,holiday,wyndham):
    string_to_list(hotel,["comment","comment_like","comment_writer"])

print(hilton['comment'][0][0])
hilton.head()# 애초에 list

❤️❤️❤️


,post_id,comment,comment_like,comment_writer,Recomment
0,1,"[❤️❤️❤️, 👏, 🤤❤️, OMG WHAT THIS IS AWESOME, I m...","[0, 0, 0, 2, 0, 0, 0, 0, 2, 3, 0, 1, 1, 1, 0, 0]","[3105070945, 17388782034, 1246744447, 11793739...","[[['@allpurple10 💙🍪', '<Profile hilton (428491..."
1,2,"[This is awesome!!!👏👏👏, Thank you!!, 👏🏻👏🏻👏🏻👏🏻👏...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[3105070945, 4464995538, 1159581929, 291908643...","[[['@allpurple10 thank you!', '<Profile hilton..."
2,3,"[Build with immortal love 👑, ❤️, 🧡🧡🧡, sooo cut...","[0, 0, 0, 0, 0]","[13811048459, 1139428551, 20534662365, 1997211...","[[], [], [], [], [['@erattray1 I saw that earl..."
3,4,[.\n.\n.\n.\n#staugustine #staugustinefl #stau...,[0],[4284913372],[[]]
4,5,[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...,"[0, 0, 0]","[4284913372, 534651664, 11230655630]","[[], [['@patriciatorressl Waldorf?', '<Profile..."


comment 첫번째에 대한 답글 -> Recomment0에서 0에 있는 것들 
comment 두번째에 대한 답글 -> Recomment0에서 1에 있는 것들 
comment 세번째에 대한 답글 -> Recomment0에서 2에 있는 것들 

document=[comment,[recomment],]

In [245]:
def string_extract(x):
    # input : 3차 
    new_list=[]
    for i in x:
        if len(i)==0:
            new_list.append([])
            
        elif len(i)==1:
            new_list.append([i[0][0]])
        
        else : 
            string_lst=[]
            for j in range(len(i)-1):
                string_lst.append(i[j][0])
            new_list.append(string_lst)
    
    return new_list

In [247]:
for hotel in (hilton,hyatt,holiday,wyndham):
    hotel["new_recom"]=hotel["Recomment"].apply(lambda x: string_extract(x))
    hotel.drop(["Recomment"],axis=1,inplace=True)

hilton.head()

,post_id,comment,comment_like,comment_writer,new_recom
0,1,"[❤️❤️❤️, 👏, 🤤❤️, OMG WHAT THIS IS AWESOME, I m...","[0, 0, 0, 2, 0, 0, 0, 0, 2, 3, 0, 1, 1, 1, 0, 0]","[3105070945, 17388782034, 1246744447, 11793739...","[[@allpurple10 💙🍪], [@hamptonrockyhill we're p..."
1,2,"[This is awesome!!!👏👏👏, Thank you!!, 👏🏻👏🏻👏🏻👏🏻👏...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[3105070945, 4464995538, 1159581929, 291908643...","[[@allpurple10 thank you!], [], [], [], [], []..."
2,3,"[Build with immortal love 👑, ❤️, 🧡🧡🧡, sooo cut...","[0, 0, 0, 0, 0]","[13811048459, 1139428551, 20534662365, 1997211...","[[], [], [], [], [@erattray1 I saw that earlie..."
3,4,[.\n.\n.\n.\n#staugustine #staugustinefl #stau...,[0],[4284913372],[[]]
4,5,[.\n.\n.\n.\n#boca #bocaraton #resortlife #exp...,"[0, 0, 0]","[4284913372, 534651664, 11230655630]","[[], [@patriciatorressl Waldorf?, @giles.ynwa ..."


new_recomment에 대한 정리는 끝    
-> 이제 comment와 new_recomment를 묶어보자. numpy가 아니기에 지금 굳이 묶을 필요는 x) ->묶으려면 아예 위계가 없게 묶는게 나은데 그건 추후에 하겠습니다.